In [1]:
from dataclasses import dataclass, field
from functools import lru_cache
from typing import NamedTuple

@lru_cache
def manhattan(a, b):
    return sum(abs(ai - bi) for ai, bi in zip(a, b))

class Point(NamedTuple):
    x: int
    y: int

@dataclass
class Sensor:
    coords: Point
    beacon: Point
    distance: int = field(init=False)
    
    def __post_init__(self):
        self.distance = manhattan(self.coords, self.beacon)

In [2]:
import re

regex_sensor = re.compile(r"(-?\d+)")

sensors = []
with open("Day15.txt") as file:
    for line in file:
        sx, sy, bx, by = map(int, regex_sensor.findall(line))
        sensors.append(Sensor(Point(sx, sy), Point(bx, by)))

In [3]:
%%time
from shapely import LineString, union_all

limit = 2_000_000
lines = []
for sensor in sensors:
    if (diff := abs(sensor.coords.y - limit)) <= sensor.distance:
        delta = abs(sensor.distance - diff)
        line = LineString(((sensor.coords.x - delta, 0), (sensor.coords.x + delta, 0)))
        lines.append(line)
int(union_all(lines).length)

CPU times: user 271 ms, sys: 414 ms, total: 686 ms
Wall time: 129 ms


5144286

In [4]:
%%time
from functools import reduce
from shapely import Polygon, difference

limit = 4_000_000
area = Polygon(((0, 0), (0, limit), (limit, limit), (limit, 0)))
zones = (
    Polygon((
        (sensor.coords.x + sensor.distance, sensor.coords.y),
        (sensor.coords.x, sensor.coords.y + sensor.distance),
        (sensor.coords.x - sensor.distance, sensor.coords.y),
        (sensor.coords.x, sensor.coords.y - sensor.distance),
    )) for sensor in sensors
)
beacon = reduce(difference, zones, area).centroid
int(beacon.x) * limit + int(beacon.y)

CPU times: user 9.09 ms, sys: 0 ns, total: 9.09 ms
Wall time: 8.07 ms


10229191267339